## DQN baselines LSTM + MLP

In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True


from agent import DQNLSTMMLPBaselineAgent
from tqdm.auto import tqdm
import random


for room_size in ["xxl"]:
    params_all = []
    for history_block_size in [1, 2, 4, 8, 16, 32]:
        for test_seed in range(5):
            num_iterations = 100 * 200
            batch_size = 32
            question_interval = 1
            terminates_at = 99

            params = {
                "env_str": "room_env:RoomEnv-v2",
                "num_iterations": num_iterations,
                "replay_buffer_size": num_iterations,
                "warm_start": num_iterations // 10,
                "batch_size": batch_size,
                "target_update_interval": 10,
                "epsilon_decay_until": num_iterations,
                "max_epsilon": 1.0,
                "min_epsilon": 0.1,
                "gamma": 0.9,
                "history_block_size": history_block_size,
                "lstm_params": {
                    "hidden_size": 96,
                    "num_layers": 2,
                    "embedding_dim": 96,
                    "bidirectional": False,
                },
                "mlp_params": {
                    "hidden_size": 96,
                    "num_hidden_layers": 1,
                    "dueling_dqn": True,
                },
                "num_samples_for_results": {"val": 5, "test": 10},
                "validation_interval": 5,
                "plotting_interval": 50,
                "train_seed": test_seed + 5,
                "test_seed": test_seed,
                "device": "cpu",
                "env_config": {
                    "question_prob": 1.0,
                    "terminates_at": terminates_at,
                    "randomize_observations": "objects",
                    "room_size": room_size,
                    "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                    "make_everything_static": False,
                    "num_total_questions": 1000,
                    "question_interval": question_interval,
                    "include_walls_in_observations": True,
                    "deterministic_objects": False,
                },
                "default_root_dir": f"./training-results/equal-object-probs/baselines/room_size={room_size}/history_block_size={history_block_size}/",
                "run_handcrafted_baselines": True,
            }
            params_all.append(params)

    for params in tqdm(params_all):
        agent = DQNLSTMMLPBaselineAgent(**params)
        agent.train()